In [1]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

# --- นาฬิกา ---
clock = pygame.time.Clock()

# --- คำถามและคำตอบ ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30

class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)

        # ตัวอักษรตรงกลางบล็อก
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)

blocks = []

def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2

    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))

def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))

def draw_wave(surface, water_level, camera_y, amplitude=10, wavelength=150, speed=2):
    """วาดเอฟเฟกต์น้ำกระเพื่อม"""
    points = []
    time_now = pygame.time.get_ticks() / 1000
    for x in range(0, WIDTH + 10, 10):
        y = (water_level - camera_y) + math.sin((x / wavelength) + time_now * speed) * amplitude
        points.append((x, y))
    if len(points) > 1:
        pygame.draw.lines(surface, LIGHT_BLUE, False, points, 3)

def main(): 
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    # --- กล้อง ---
    camera_y = 0
    tower_height = 0
    camera_box_top = HEIGHT / 3       # ขอบบนของกล่องกล้อง
    camera_box_bottom = HEIGHT * 2/3  # ขอบล่างของกล้อง
    camera_target_y = 0

    # --- น้ำ ---
    water_level = HEIGHT - 100
    base_rise_speed = 0.3          # น้ำขึ้นช้าลง
    water_speed = base_rise_speed
    max_rise_speed = 0.4            # จำกัดความเร็วสูงสุดของน้ำ

    # --- คะแนน ---
    score = 0

    # --- Game Over ---
    game_over = False

    while running:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        if input_text.lower() in current_question:
                            add_word_blocks(input_text)
                            tower_height = len(blocks) * block_height
                            score += len(input_text)
                            input_text = ""
                            question = random.choice(list(questions.keys()))
                            current_question = questions[question]
                        else:
                            input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode
                else:
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                        water_level = HEIGHT - 100
                        water_speed = base_rise_speed
                        tower_height = 0
                        camera_y = 0
                        score = 0
                        game_over = False

        # --- อัปเดตกล้องแบบ Box ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            top_view = top_block_y - camera_y
            bottom_view = bottom_block_y - camera_y

            # ถ้าเกินขอบบน/ล่างกล่อง ให้ขยับกล้อง
            if top_view < camera_box_top:
                camera_target_y = top_block_y - camera_box_top
            elif bottom_view > camera_box_bottom:
                camera_target_y = bottom_block_y - camera_box_bottom
        else:
            camera_target_y = 0

        # กล้องขยับนุ่มนวล
        camera_y += (camera_target_y - camera_y) * 0.08

        # --- น้ำขึ้นแบบค่อยเป็นค่อยไป ---
        # เพิ่มช้ากว่าก่อน และจำกัดความเร็วสูงสุด
        water_speed = min(base_rise_speed + (tower_height / 10000), max_rise_speed)
        water_level -= water_speed

        # --- ตรวจสอบ Game Over ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if water_level <= top_block_y + block_height:
                game_over = True

        # --- วาดบล็อก ---
        for block in blocks:
            block.draw(screen, camera_y)

        # --- วาดน้ำ ---
        water_height = max(1, HEIGHT - int(water_level - camera_y))  # ป้องกันค่าติดลบ
        water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
        water_surface.fill(BLUE)
        screen.blit(water_surface, (0, water_level - camera_y))
        draw_wave(screen, water_level, camera_y)

        # --- อินพุต & คะแนน ---
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)

        # --- Game Over ---
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 18:33:43.491 python[48991:12664555] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


In [1]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

# --- นาฬิกา ---
clock = pygame.time.Clock()

WATER_ANIM_DURATION = 0.9  # ระยะเวลาเคลื่อนขึ้นของน้ำ
WATER_SURFACE_WAVE_AMPLITUDE = 6
WATER_SURFACE_LENGTH = 140 # ความยาวคลื่นผิวน้ำ
round_number = 1
PERCENT_START = 0.50
PERCENT_STEP = 0.05 
PERCENT_MAX = 1.0      

# --- คำถามและคำตอบ ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30

class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)

        # ตัวอักษรตรงกลางบล็อก
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)

blocks = []

def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2

    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))

def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))

class Water:
    def __init__(self, screen_h):
        self.screen_h = screen_h
        self.level_y = float(screen_h)
        self.visible = False
        self.animating = False
        self.start_y = self.level_y 
        self.target_y = self.level_y
        self.anim_time = 0.0
        self.anim_dur = WATER_ANIM_DURATION

        # เก็บความยาวคำทั้งหมด (สะสม)
        self.word_lengths = []

    def start_rise(self, rise_pixels):
        if not self.visible:
            self.visible = True
        self.start_y = float(self.level_y)
        self.target_y = max(0.0, self.start_y - rise_pixels) # น้ำสามารถขึ้นไปจนถึง 0.0
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        if self.animating:
            self.anim_time += dt
            t = min(1.0, self.anim_time / self.anim_dur)
            t_ease = t * t * (3 - 2 * t)
            self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease
            if t >= 1.0:
                self.level_y = self.target_y
                self.animating = False

    def draw(self, surf):
        if not self.visible:
            return
        level = int(self.level_y)
        if level < self.screen_h:
            # วาดผืนน้ำเป็น surface โปร่งแสง
            water_surface = pygame.Surface((WIDTH, self.screen_h - level), pygame.SRCALPHA)
            water_surface.fill(WATER_COLOR)
            surf.blit(water_surface, (0, level))

            # วาดเส้นคลื่นด้านบน
            points = []
            t = pygame.time.get_ticks() / 1000.0
            amp = WATER_SURFACE_WAVE_AMPLITUDE
            length = WATER_SURFACE_LENGTH
            for x in range(0, WIDTH + 8, 8):
                y = level + math.sin((x / length) + t * 1.5) * amp
                points.append((x, y))
            if points:
                pygame.draw.lines(surf, (255, 255, 255), False, points, 3)

    def is_animating(self):
        return self.animating

    def reset(self):
        # รีเซ็ตสถานะทั้งหมด
        self.__init__(self.screen_h)


def main(): 
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    # --- กล้อง ---
    camera_y = 0
    tower_height = 0
    camera_box_height = HEIGHT // 2  # ความสูงกล้องที่มองได้
    camera_target_y = 0

    # --- น้ำ ---
    water_level = HEIGHT - 100
    base_rise_speed = 0.1
    water_speed = base_rise_speed

    # --- คะแนน ---
    score = 0

    # --- Game Over ---
    game_over = False

        # ก่อน while loop ใน main()
    water = Water(HEIGHT)
    
    # ตัวแปรควบคุมเปอร์เซ็นต์การขึ้นน้ำ
    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.05
    PERCENT_MAX = 1.0


    while running:
            dt = clock.tick(60) / 1000.0
    
            # --- 1. จัดการ Events ---
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                if game_over:
                    if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                        player.reset()
                        water.reset()
                        scoring.reset()
                        typer.clear_input()
                        game_over = False
                        game_won = False
                        final_score_data = {}
                else:
                    # ให้ typer เก็บ input ตามปกติ (ไม่ต้องเรียกซ้ำถ้าน้ำกำลังขึ้น)
                    submitted = typer.handle_event(event)
    
                    # ถ้ายังไม่มี animation ของน้ำ (น้ำไม่กำลังขึ้น) ให้ประมวลผลการ submit
                    if submitted and not water.is_animating():
                        is_correct = typer.check_answer()
                        letters = typer.get_answer_length()  # ความยาวคำที่พิมพ์
                        # default
                        # (เราจะใช้ water.word_lengths เป็น list สะสมของความยาวคำแล้วคำนวณ average)
                        
                        if is_correct:
                            # บันทึกผลคะแนนและเพิ่มความสูงบล็อกให้ผู้เล่น
                            scoring.register_word(typer.user_input)
                            player.add_height(letters)
    
                            # ถ้าขึ้นชนเพดาน -> ชนะทันที (ไม่สั่งน้ำขึ้น)
                            if player.get_top_y() <= 0:
                                game_over = True
                                game_won = True
                                final_score_data = scoring.game_over()
                                typer.clear_input()
                                # ถ้าต้องการให้นับคำรอบชนะ ให้ uncomment บรรทัดด้านล่าง
                                # water.word_lengths.append(letters)
                                # round_number += 1
                                continue
    
                            # ถ้าตอบถูก ให้บันทึกความยาวคำเพื่อใช้คำนวณเฉลี่ย
                            water.word_lengths.append(letters)
                        else:
                            # ถ้าตอบผิด: ถ้าต้องการให้นับคำผิดใน avg ให้ append ด้วย
                            # ถ้าไม่ต้องการแสดงผลต่อ avg ให้คอมเมนต์บรรทัดต่อไป
                            water.word_lengths.append(letters)
                            # ไม่เพิ่มบล็อกให้ผู้เล่น
    
                        # คำนวณ percent ตามรอบ: 50%, 55%, 60% ...
                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                        percent = min(percent, PERCENT_MAX)
    
                        # ค่าเฉลี่ยสะสม (รวมคำปัจจุบันที่ append แล้ว)
                        if len(water.word_lengths) > 0:
                            avg_len = sum(water.word_lengths) / len(water.word_lengths)
                        else:
                            avg_len = 0.0
    
                        # จำนวนตัวอักษรที่น้ำจะขึ้น (เป็น float)
                        water_rise_letters = avg_len * percent
    
                        # สั่งให้น้ำขึ้น หากเกมยังไม่จบ
                        if water_rise_letters > 0 and not game_over:
                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                            water.start_rise(rise_px)
    
                        # เตรียมรอบถัดไป
                        round_number += 1
    
                        # เคลียร์อินพุตของผู้เล่น
                        typer.clear_input()
    
            # --- 2. อัปเดตสถานะเกม ---
            if not game_over:
                # อัปเดตอนิเมชันของน้ำ (ใช้ dt เพื่อให้สมูท)
                water.update(dt)
                
                # ตรวจสอบ Game Over (จมน้ำ) — ให้เช็คเมื่ออนิเมชันน้ำจบ (น้ำอยู่ที่ตำแหน่งสุดท้าย)
                if (not water.is_animating()) and water.visible and (water.level_y <= player.get_top_y()):
                    game_over = True
                    game_won = False
                    final_score_data = scoring.game_over()
    
            # --- อัปเดตกล้องแบบ Box ---
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                bottom_block_y = max(block.y for block in blocks)
                tower_top_in_view = top_block_y - camera_y
                tower_bottom_in_view = bottom_block_y - camera_y
    
                # ถ้า tower เลยกรอบบน/ล่าง ให้กล้องขยับ
                if tower_top_in_view < HEIGHT / 4:
                    camera_target_y = top_block_y - HEIGHT / 4
                elif tower_bottom_in_view > HEIGHT * 3 / 4:
                    camera_target_y = bottom_block_y - HEIGHT * 3 / 4
            else:
                camera_target_y = 0
    
            camera_y += (camera_target_y - camera_y) * 0.1  # smooth move
    
            # --- น้ำขึ้น ---
            water_speed = base_rise_speed + (tower_height / 5000)
            water_level -= water_speed
    
            # --- ตรวจสอบ Game Over ---
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                if water_level <= top_block_y:
                    game_over = True
    
            # --- วาดบล็อก ---
            for block in blocks:
                block.draw(screen, camera_y)
    
            # --- วาดน้ำ ---
            water_height = max(1, HEIGHT - int(water_level - camera_y))  # ป้องกันค่าติดลบ
            water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
            water_surface.fill(BLUE)
            screen.blit(water_surface, (0, water_level - camera_y))
            draw_wave(screen, water_level, camera_y)
    
            # --- กล่อง input ---
            input_box = pygame.Rect(20, 20, 760, 50)
            pygame.draw.rect(screen, WHITE, input_box, 2)
            draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
            draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
    
            # --- คะแนน ---
            draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)
    
            # --- Game Over ---
            if game_over:
                overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
                overlay.fill((0, 0, 0, 180))
                screen.blit(overlay, (0, 0))
                draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
                draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
                draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)
    
            pygame.display.flip()
            clock.tick(60)
    
    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'typer' is not defined

In [1]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30

class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)

blocks = []

def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))

def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))

def draw_wave(surface, water_y, camera_y):
    t = pygame.time.get_ticks() / 300.0
    points = []
    for x in range(0, WIDTH, 8):
        y = water_y - camera_y + math.sin(x / 40 + t) * 5
        points.append((x, y))
    pygame.draw.lines(surface, WHITE, False, points, 2)

def main():
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    camera_y = 0
    camera_target_y = 0

    # --- น้ำ ---
    water_level = HEIGHT - 100
    water_target = water_level
    water_speed = 150  # ความเร็วในการขึ้น (px/วินาที)

    score = 0
    game_over = False

    while running:
        dt = clock.tick(60) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if game_over:
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    blocks.clear()
                    water_level = HEIGHT - 100
                    water_target = water_level
                    input_text = ""
                    question = random.choice(list(questions.keys()))
                    current_question = questions[question]
                    score = 0
                    game_over = False
            else:
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RETURN:
                        if input_text.lower() in current_question:
                            add_word_blocks(input_text)
                            score += len(input_text) * 10
                            # 🌊 น้ำขึ้นตามความยาวคำ
                            water_target -= len(input_text) * 10
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode

        if not game_over:
            # 🌊 อัปเดตน้ำ (ลอยขึ้นแบบ smooth)
            if water_level > water_target:
                water_level -= water_speed * dt
                if water_level < water_target:
                    water_level = water_target

            # --- กล้อง ---
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                bottom_block_y = max(block.y for block in blocks)

                if top_block_y - camera_y < HEIGHT / 4:
                    camera_target_y = top_block_y - HEIGHT / 4
                elif bottom_block_y - camera_y > HEIGHT * 3 / 4:
                    camera_target_y = bottom_block_y - HEIGHT * 3 / 4
            else:
                camera_target_y = 0

            camera_y += (camera_target_y - camera_y) * 0.1

            # 🔥 ตรวจสอบเมื่อน้ำถึงบล็อกบนสุด
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                if water_level <= top_block_y + block_height:
                    game_over = True

        # --- วาดพื้นหลัง ---
        screen.fill(BLACK)

        # --- วาดบล็อก ---
        for block in blocks:
            block.draw(screen, camera_y)

        # --- วาดน้ำ ---
        water_height = max(1, HEIGHT - int(water_level - camera_y))
        water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
        water_surface.fill(BLUE)
        screen.blit(water_surface, (0, water_level - camera_y))
        draw_wave(screen, water_level, camera_y)

        # --- UI ---
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)

        # --- Game Over ---
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 18:46:43.516 python[49376:12677023] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


In [1]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- คลาสน้ำ ---
class Water:
    def __init__(self, start_level, speed=150, color=BLUE):
        self.level = start_level         # ระดับน้ำปัจจุบัน
        self.target = start_level        # ระดับน้ำเป้าหมาย
        self.speed = speed               # ความเร็วในการขึ้น
        self.color = color

    def rise(self, amount):
        """ให้น้ำขึ้นไปอีกตามจำนวนพารามิเตอร์"""
        self.target -= amount

    def update(self, dt):
        """อัปเดตตำแหน่งน้ำให้ค่อยๆ ขยับไปหา target"""
        if self.level > self.target:
            self.level -= self.speed * dt
            if self.level < self.target:
                self.level = self.target

    def draw(self, surface, camera_y):
        """วาดพื้นผิวน้ำและคลื่น"""
        # น้ำ
        water_height = max(1, HEIGHT - int(self.level - camera_y))
        water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
        water_surface.fill(self.color)
        surface.blit(water_surface, (0, self.level - camera_y))

        # คลื่น
        self.draw_wave(surface, camera_y)

    def draw_wave(self, surface, camera_y):
        """วาดคลื่นน้ำ"""
        t = pygame.time.get_ticks() / 300.0
        points = []
        for x in range(0, WIDTH, 8):
            y = self.level - camera_y + math.sin(x / 40 + t) * 5
            points.append((x, y))
        pygame.draw.lines(surface, WHITE, False, points, 2)


# --- ฟังก์ชันอื่น ๆ ---
blocks = []


def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    camera_y = 0
    camera_target_y = 0

    # 🌊 สร้างอ็อบเจกต์น้ำ
    water = Water(start_level=HEIGHT - 100)

    score = 0
    game_over = False

    while running:
        dt = clock.tick(60) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if game_over:
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    blocks.clear()
                    water = Water(start_level=HEIGHT - 100)
                    input_text = ""
                    question = random.choice(list(questions.keys()))
                    current_question = questions[question]
                    score = 0
                    game_over = False
            else:
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RETURN:
                        if input_text.lower() in current_question:
                            add_word_blocks(input_text)
                            score += len(input_text) * 10
                            # 🌊 น้ำขึ้นตามความยาวคำ
                            water.rise(len(input_text) * 10)
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode

        if not game_over:
            # 🌊 อัปเดตน้ำ
            water.update(dt)

            # --- กล้อง ---
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                bottom_block_y = max(block.y for block in blocks)

                if top_block_y - camera_y < HEIGHT / 4:
                    camera_target_y = top_block_y - HEIGHT / 4
                elif bottom_block_y - camera_y > HEIGHT * 3 / 4:
                    camera_target_y = bottom_block_y - HEIGHT * 3 / 4
            else:
                camera_target_y = 0

            camera_y += (camera_target_y - camera_y) * 0.1

            # 🔥 ตรวจสอบเมื่อน้ำถึงบล็อกบนสุด
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                if water.level <= top_block_y + block_height:
                    game_over = True

        # --- วาดพื้นหลัง ---
        screen.fill(BLACK)

        # --- วาดบล็อก ---
        for block in blocks:
            block.draw(screen, camera_y)

        # --- วาดน้ำ ---
        water.draw(screen, camera_y)

        # --- UI ---
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)

        # --- Game Over ---
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 18:49:36.954 python[49452:12680249] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


In [ ]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- คลาสน้ำ ---
class Water:
    def __init__(self, start_level, speed=150, color=BLUE):
        self.level = start_level         # ระดับน้ำปัจจุบัน
        self.target = start_level        # ระดับน้ำเป้าหมาย
        self.speed = speed               # ความเร็วในการขึ้น
        self.color = color

    def rise(self, amount):
        """ให้น้ำขึ้นไปอีกตามจำนวนพารามิเตอร์"""
        self.target -= amount

    def update(self, dt):
        """อัปเดตตำแหน่งน้ำให้ค่อยๆ ขยับไปหา target"""
        if self.level > self.target:
            self.level -= self.speed * dt
            if self.level < self.target:
                self.level = self.target

    def draw(self, surface, camera_y):
        """วาดพื้นผิวน้ำและคลื่น"""
        # น้ำ
        water_height = max(1, HEIGHT - int(self.level - camera_y))
        water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
        water_surface.fill(self.color)
        surface.blit(water_surface, (0, self.level - camera_y))

        # คลื่น
        self.draw_wave(surface, camera_y)

    def draw_wave(self, surface, camera_y):
        """วาดคลื่นน้ำ"""
        t = pygame.time.get_ticks() / 300.0
        points = []
        for x in range(0, WIDTH, 8):
            y = self.level - camera_y + math.sin(x / 40 + t) * 5
            points.append((x, y))
        pygame.draw.lines(surface, WHITE, False, points, 2)


# --- ฟังก์ชันอื่น ๆ ---
blocks = []


def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    camera_y = 0
    camera_target_y = 0

    # 🌊 สร้างอ็อบเจกต์น้ำ
    water = Water(start_level=HEIGHT - 100)

    score = 0
    game_over = False

    while running:
        dt = clock.tick(60) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if game_over:
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    blocks.clear()
                    water = Water(start_level=HEIGHT - 100)
                    input_text = ""
                    question = random.choice(list(questions.keys()))
                    current_question = questions[question]
                    score = 0
                    game_over = False
            else:
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RETURN:
                        if input_text.lower() in current_question:
                            add_word_blocks(input_text)
                            score += len(input_text) * 10
                            # 🌊 น้ำขึ้นตามความยาวคำ
                            water.rise(len(input_text) * 10)
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode

        if not game_over:
            # 🌊 อัปเดตน้ำ
            water.update(dt)

            # --- กล้อง ---
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                bottom_block_y = max(block.y for block in blocks)

                if top_block_y - camera_y < HEIGHT / 4:
                    camera_target_y = top_block_y - HEIGHT / 4
                elif bottom_block_y - camera_y > HEIGHT * 3 / 4:
                    camera_target_y = bottom_block_y - HEIGHT * 3 / 4
            else:
                camera_target_y = 0

            camera_y += (camera_target_y - camera_y) * 0.1

            # 🔥 ตรวจสอบเมื่อน้ำถึงบล็อกบนสุด
            if len(blocks) > 0:
                top_block_y = min(block.y for block in blocks)
                if water.level <= top_block_y + block_height:
                    game_over = True

        # --- วาดพื้นหลัง ---
        screen.fill(BLACK)

        # --- วาดบล็อก ---
        for block in blocks:
            block.draw(screen, camera_y)

        # --- วาดน้ำ ---
        water.draw(screen, camera_y)

        # --- UI ---
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)

        # --- Game Over ---
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()

In [ ]:
import pygame import math # (ลบ import random ออก) # --- ค่าคงที่ของเกม --- WIDTH, HEIGHT = 800, 600 BLOCK_SIZE = 20 BACKGROUND_COLOR = (10, 20, 40) WATER_COLOR = (20, 80, 200, 200) SURFACE_HIGHLIGHT = (100, 150, 255, 100) WATER_ANIM_DURATION = 0.9 WATER_SURFACE_WAVE_AMPLITUDE = 6 WATER_SURFACE_LENGTH = 140 round_number = 1 PERCENT_START = 0.50 PERCENT_STEP = 0.05 PERCENT_MAX = 1.0 WATER_RISE_PIXELS_PER_LETTER = 10 # เพิ่มให้ครบ # --- คลาส typingText --- class typingText: def __init__(self, screen, font, question, valid_answers): self.screen = screen self.font = font self.question = question self.valid_answers = set(ans.lower() for ans in valid_answers) self.user_input = "" self.message = "" self.message_color = (255, 255, 255) def handle_event(self, event): submitted = False if event.type == pygame.KEYDOWN: if event.key == pygame.K_BACKSPACE: self.user_input = self.user_input[:-1] elif event.key == pygame.K_RETURN: if self.user_input: submitted = True else: char = event.unicode if char.isprintable(): self.user_input += char return submitted def check_answer(self): is_correct = self.user_input.lower() in self.valid_answers if is_correct: self.message = "Correct!" self.message_color = (0, 255, 0) else: self.message = "Invalid or Unknown Word!" self.message_color = (255, 0, 0) return is_correct def get_answer_length(self): return len(self.user_input) def clear_input(self): self.user_input = "" self.message = "" def draw(self): question_surf = self.font.render(f"Question: {self.question}", True, (255, 255, 255)) self.screen.blit(question_surf, (20, 20)) input_rect = pygame.Rect(20, 60, WIDTH - 40, 50) pygame.draw.rect(self.screen, (255, 255, 255), input_rect, 2) input_surf = self.font.render(self.user_input, True, (255, 255, 0)) self.screen.blit(input_surf, (input_rect.x + 10, input_rect.y + 10)) result_surf = self.font.render(self.message, True, self.message_color) self.screen.blit(result_surf, (20, 120)) # --- คลาส Water --- class Water: def __init__(self, screen_h): self.screen_h = screen_h self.level_y = float(screen_h) self.visible = False self.animating = False self.start_y = self.level_y self.target_y = self.level_y self.anim_time = 0.0 self.anim_dur = WATER_ANIM_DURATION self.word_lengths = [] def start_rise(self, rise_pixels): if not self.visible: self.visible = True self.start_y = float(self.level_y) self.target_y = max(0.0, self.start_y - rise_pixels) self.anim_time = 0.0 self.animating = True def update(self, dt): if self.animating: self.anim_time += dt t = min(1.0, self.anim_time / self.anim_dur) t_ease = t * t * (3 - 2 * t) self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease if t >= 1.0: self.level_y = self.target_y self.animating = False def draw(self, surf, camera_y=0): if not self.visible: return level = int(self.level_y - camera_y) if level < self.screen_h: water_surface = pygame.Surface((WIDTH, self.screen_h - level), pygame.SRCALPHA) water_surface.fill(WATER_COLOR) surf.blit(water_surface, (0, level)) points = [] t = pygame.time.get_ticks() / 1000.0 amp = WATER_SURFACE_WAVE_AMPLITUDE length = WATER_SURFACE_LENGTH for x in range(0, WIDTH + 8, 8): y = level + math.sin((x / length) + t * 1.5) * amp points.append((x, y)) if points: pygame.draw.lines(surf, (255, 255, 255), False, points, 3) def is_animating(self): return self.animating def reset(self): self.__init__(self.screen_h) # --- คลาส Scoring --- class Scoring: def __init__(self): self.char_score = 0 self.word_score = 0 self.words_history = [] self.is_game_over = False def register_word(self, word: str): if self.is_game_over: return clean_word = word.strip() if not clean_word: return self.char_score += len(clean_word) self.word_score += 1 self.words_history.append(clean_word) def game_over(self): self.is_game_over = True return self.get_final_score() def get_final_score(self): total = self.char_score + self.word_score return { "char_score": self.char_score, "word_score": self.word_score, "total_score": total, "words_history": self.words_history, } def reset(self): self.char_score = 0 self.word_score = 0 self.words_history = [] self.is_game_over = False # --- คลาส Player --- class Player: def __init__(self, screen_w, screen_h, block_size): self.screen_w = screen_w self.screen_h = screen_h self.block_size = block_size self.width = 100 self.x_pos = (self.screen_w // 2) - (self.width // 2) self.color = (200, 200, 200) self.survivor_color = (255, 0, 0) self.total_height_pixels = self.block_size self.y_pos = self.screen_h - self.total_height_pixels def add_height(self, num_letters): self.total_height_pixels += num_letters * self.block_size self.y_pos = self.screen_h - self.total_height_pixels def get_top_y(self): return (self.screen_h - self.total_height_pixels) - 20 def draw(self, surf, camera_y=0): draw_height = min(self.total_height_pixels, self.screen_h * 3) draw_y = self.screen_h - draw_height - camera_y player_rect = pygame.Rect(self.x_pos, draw_y, self.width, draw_height) pygame.draw.rect(surf, self.color, player_rect) survivor_y = self.get_top_y() - camera_y survivor_rect = pygame.Rect(self.x_pos + (self.width // 2) - 10, survivor_y, 20, 20) pygame.draw.rect(surf, self.survivor_color, survivor_rect) def reset(self): self.__init__(self.screen_w, self.screen_h, self.block_size) # --- ฟังก์ชันหลัก --- def main(): pygame.init() screen = pygame.display.set_mode((WIDTH, HEIGHT)) pygame.display.set_caption("Text or Die - Camera Mode") main_font = pygame.font.Font(None, 40) score_font = pygame.font.Font(None, 50) small_font = pygame.font.Font(None, 24) clock = pygame.time.Clock() valid_answers = [ "apple", "banana", "orange", "grape", "mango", "melon", "kiwi", "pear", "peach", "plum", "cherry", "berry", "lime", "lemon" ] question = "Name a fruit" player = Player(WIDTH, HEIGHT, BLOCK_SIZE) water = Water(HEIGHT) scoring = Scoring() typer = typingText(screen, main_font, question, valid_answers) running = True game_over = False game_won = False final_score_data = {} # --- ตัวแปรกล้อง --- camera_y = 0.0 camera_target_y = 0.0 while running: dt = clock.tick(60) / 1000.0 for event in pygame.event.get(): if event.type == pygame.QUIT: running = False if game_over: if event.type == pygame.KEYDOWN and event.key == pygame.K_r: player.reset() water.reset() scoring.reset() typer.clear_input() game_over = False game_won = False final_score_data = {} camera_y = 0.0 camera_target_y = 0.0 else: if not water.is_animating(): submitted = typer.handle_event(event) if submitted: is_correct = typer.check_answer() letters = typer.get_answer_length() water_rise_letters = 0.0 if is_correct: scoring.register_word(typer.user_input) player.add_height(letters) water_rise_letters = letters / 2.0 if player.get_top_y() <= 0: game_over = True game_won = True final_score_data = scoring.game_over() typer.clear_input() continue water.word_lengths.append(letters) else: water.word_lengths.append(letters) percent = PERCENT_START + PERCENT_STEP * (round_number - 1) percent = min(percent, PERCENT_MAX) if len(water.word_lengths) > 0: avg_len = sum(water.word_lengths) / len(water.word_lengths) else: avg_len = 0.0 water_rise_letters = avg_len * percent if water_rise_letters > 0 and not game_over: rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER water.start_rise(rise_px) typer.clear_input() if not game_over: water.update(dt) if not water.is_animating() and water.visible and water.level_y <= player.get_top_y(): game_over = True game_won = False final_score_data = scoring.game_over() # --- อัปเดตตำแหน่งกล้อง --- player_top = player.get_top_y() # กล้องจะเลื่อนตามหัวผู้เล่น โดยมีระยะเว้นข้างบน 200px camera_target_y = max(0, -player_top + 200) camera_y += (camera_target_y - camera_y) * 5.0 * dt # smooth follow # --- วาด --- screen.fill(BACKGROUND_COLOR) player.draw(screen, camera_y) water.draw(screen, camera_y) typer.draw() if game_over: overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA) overlay.fill((0, 0, 0, 180)) screen.blit(overlay, (0, 0)) if game_won: go_text = score_font.render("YOU REACHED THE TOP!", True, (0, 255, 100)) else: go_text = score_font.render("GAME OVER", True, (255, 0, 0)) screen.blit(go_text, (WIDTH // 2 - go_text.get_width() // 2, HEIGHT // 3)) restart_text = small_font.render("Press 'R' to Restart", True, (150, 150, 150)) screen.blit(restart_text, (WIDTH // 2 - restart_text.get_width() // 2, HEIGHT - 50)) pygame.display.flip() pygame.quit() if __name__ == "__main__": main()

In [ ]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- คลาสน้ำ (เวอร์ชันเดิมของคุณ) ---
class Water:
    def __init__(self, start_level, speed=150, color=BLUE):
        self.level = start_level         # ระดับน้ำปัจจุบัน (world y)
        self.target = start_level        # ระดับน้ำเป้าหมาย (world y)
        self.speed = speed               # ความเร็วในการขึ้น (px/s)
        self.color = color
        # เก็บความยาวคำ (จะถูกสร้างจาก main เมื่อสร้าง water)
        # self.word_lengths = []

    def rise(self, amount):
        """ให้น้ำขึ้นไปอีกตามจำนวนพารามิเตอร์ (amount เป็นพิกเซล)"""
        self.target -= amount

    def update(self, dt):
        """อัปเดตตำแหน่งน้ำให้ค่อยๆ ขยับไปหา target"""
        if self.level > self.target:
            self.level -= self.speed * dt
            if self.level < self.target:
                self.level = self.target
        

    def draw(self, surface, camera_y):
        """วาดพื้นผิวน้ำและคลื่น"""
        # คำนวณความสูงของพื้นผิวน้ำในหน้าจอ (screen space)
        screen_y = int(self.level - camera_y)
        water_height = max(0, HEIGHT - screen_y)
        if water_height > 0:
            water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
            # fill with RGBA color if a 4-tuple, else fallback
            try:
                water_surface.fill(self.color)
            except Exception:
                water_surface.fill((30, 70, 200, 180))
            surface.blit(water_surface, (0, screen_y))

            # คลื่น
            self.draw_wave(surface, screen_y)

    def draw_wave(self, surface, screen_level):
        """วาดคลื่น (screen_level เป็น y ใน screen space)"""
        t = pygame.time.get_ticks() / 300.0
        points = []
        for x in range(0, WIDTH, 8):
            y = screen_level + math.sin(x / 40 + t) * 5
            points.append((x, y))
        if len(points) > 1:
            pygame.draw.lines(surface, WHITE, False, points, 2)


# --- ฟังก์ชันอื่น ๆ ---
blocks = []


def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง (stack)"""
    word = word.strip()
    if word == "":
        return

    # base top y for the lowest new block:
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height  # top y of the first (lowest) block
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    camera_y = 0
    camera_target_y = 0

    # 🌊 สร้างอ็อบเจกต์น้ำ (และ attach list สำหรับความยาวคำ)
    water = Water(start_level=HEIGHT - 100)
    water.word_lengths = []  # สตอร์ความยาวคำสะสม

    # ตั้งค่าการเพิ่มระดับน้ำ
    WATER_RISE_PIXELS_PER_LETTER = block_height  # 1 ตัวอักษร = 1 บล็อกสูง
    # รอบ / percent
    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10
    PERCENT_MAX = float('inf')

    score = 0
    game_over = False

    while running:
        dt = clock.tick(60) / 1000.0

        # --- Events: ใช้บล็อกที่คุณให้ (ปรับให้เข้ากับเมธอด water.rise) ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # correct?
                            if submitted_word in current_question:
                                # correct -> add blocks and score
                                add_word_blocks(submitted_word)
                                tower_height = len(blocks) * block_height
                                score += len(submitted_word) * 10

                                # record length for avg
                                water.word_lengths.append(len(submitted_word))

                                # calculate percent & rise
                                percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                percent = min(percent, PERCENT_MAX)
                                avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                water_rise_letters = avg_len * percent
                                if water_rise_letters > 0:
                                    rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                    # call water.rise (your Water uses rise(amount in px))
                                    water.rise(rise_px)

                                round_number += 1

                                # new question
                                question = random.choice(list(questions.keys()))
                                current_question = questions[question]
                            else:
                                # incorrect -> don't add blocks, but count length for water and cause water to rise
                                wrong_len = len(submitted_word)
                                # include wrong answers in avg (comment out next line if you don't want that)
                                water.word_lengths.append(wrong_len)

                                percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                percent = min(percent, PERCENT_MAX)
                                avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                water_rise_letters = avg_len * percent
                                if water_rise_letters > 0:
                                    rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                    water.rise(rise_px)

                                round_number += 1

                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    # รีสตาร์ทเมื่อ game_over
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                        water = Water(start_level=HEIGHT - 100)
                        water.word_lengths = []
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False

        # --- อัปเดตน้ำ ---
        water.update(dt)

        # --- กล้อง (box follow) ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.12

        # --- ตรวจสอบ Game Over: เมื่อน้ำถึงบล็อกบนสุด (เช็คเมื่ออนิเมชัน water ถึง target แล้ว) ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if (not (water.level > water.target)) and (water.level <= top_block_y):
                game_over = True

        # --- วาดฉาก ---
        screen.fill(BLACK)

        # draw blocks
        for block in blocks:
            block.draw(screen, camera_y)

        # draw water
        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 200, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 200, 80)

        # game over overlay
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 19:32:17.989 Python[36044:13053216] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

In [ ]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 28)
BIG = pygame.font.SysFont("arial", 56)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- คลาสน้ำ (เวอร์ชันเดิมของคุณ) ---
class Water:
    def __init__(self, start_level, speed=150, color=BLUE):
        self.level = start_level         # ระดับน้ำปัจจุบัน (world y)
        self.target = start_level        # ระดับน้ำเป้าหมาย (world y)
        self.speed = speed               # ความเร็วในการขึ้น (px/s)
        self.color = color
        # เก็บความยาวคำ (จะถูกสร้างจาก main เมื่อสร้าง water)
        self.word_lengths = []

    def rise(self, amount):
        """ให้น้ำขึ้นไปอีกตามจำนวนพารามิเตอร์ (amount เป็นพิกเซล)"""
        self.target -= amount

    def update(self, dt):
        """อัปเดตตำแหน่งน้ำให้ค่อยๆ ขยับไปหา target"""
        if self.level > self.target:
            self.level -= self.speed * dt
            if self.level < self.target:
                self.level = self.target

    def draw(self, surface, camera_y):
        """วาดพื้นผิวน้ำและคลื่น"""
        # คำนวณความสูงของพื้นผิวน้ำในหน้าจอ (screen space)
        screen_y = int(self.level - camera_y)
        water_height = max(0, HEIGHT - screen_y)
        if water_height > 0:
            water_surface = pygame.Surface((WIDTH, water_height), pygame.SRCALPHA)
            # fill with RGBA color if a 4-tuple, else fallback
            try:
                water_surface.fill(self.color)
            except Exception:
                water_surface.fill((30, 70, 200, 180))
            surface.blit(water_surface, (0, screen_y))

            # คลื่น
            self.draw_wave(surface, screen_y)

    def draw_wave(self, surface, screen_level):
        """วาดคลื่น (screen_level เป็น y ใน screen space)"""
        t = pygame.time.get_ticks() / 300.0
        points = []
        for x in range(0, WIDTH, 8):
            y = screen_level + math.sin(x / 40 + t) * 5
            points.append((x, y))
        if len(points) > 1:
            pygame.draw.lines(surface, WHITE, False, points, 2)


# --- ฟังก์ชันอื่น ๆ ---
blocks = []


def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง (stack)"""
    word = word.strip()
    if word == "":
        return

    # base top y for the lowest new block:
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height  # top y of the first (lowest) block
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""
    # เลือกหมวดแรก (สุ่มหนึ่งหมวดให้เล่นเป็นรอบ)
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    camera_y = 0
    camera_target_y = 0

    # 🌊 สร้างอ็อบเจกต์น้ำ (และ attach list สำหรับความยาวคำ)
    water = Water(start_level=HEIGHT - 100)

    # ตั้งค่าการเพิ่มระดับน้ำ
    WATER_RISE_PIXELS_PER_LETTER = block_height  # 1 ตัวอักษร = 1 บล็อกสูง
    # รอบ / percent (ไม่จำกัดสูงสุดตามคำขอ)
    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10
    # no PERCENT_MAX (ไม่ cap)

    score = 0
    game_over = False

    # เก็บคำที่ผู้เล่นพิมพ์มาแล้ว (lowercase) เพื่อกันคำซ้ำ
    submitted_words = set()

    # ข้อความแสดงชั่วคราว
    info_text = ""
    info_time = 0.0
    INFO_DURATION = 2.0  # วินาที

    while running:
        dt = clock.tick(60) / 1000.0

        # เคลียร์ข้อความชั่วคราวเมื่อครบเวลา
        if info_text and (pygame.time.get_ticks() / 1000.0 - info_time) > INFO_DURATION:
            info_text = ""

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # 1) ตรวจคำซ้ำก่อน
                            if submitted_word in submitted_words:
                                info_text = "Already used!"  # duplicate
                                info_time = pygame.time.get_ticks() / 1000.0
                            else:
                                # ถ้ายังไม่ซ้ำ ให้ตรวจคำถูกหรือผิดกับ current_question
                                if submitted_word in current_question:
                                    # ถูกต้อง
                                    submitted_words.add(submitted_word)  # บันทึกคำนี้
                                    add_word_blocks(submitted_word)
                                    tower_height = len(blocks) * block_height
                                    score += len(submitted_word) * 10

                                    # record length for avg
                                    water.word_lengths.append(len(submitted_word))

                                    # calculate percent & rise (no cap)
                                    percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                    water_rise_letters = avg_len * percent
                                    if water_rise_letters > 0:
                                        rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                        water.rise(rise_px)

                                    round_number += 1

                                    # เปลี่ยนหมวดสำหรับรอบถัดไป (สุ่มหมวดใหม่ 1 หมวดต่อรอบ)
                                    question = random.choice(list(questions.keys()))
                                    current_question = questions[question]

                                    info_text = "Correct!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                else:
                                    # ผิด → ไม่เพิ่มบล็อก
                                    info_text = "Wrong!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    # ถ้าผิดในรอบแรก ให้ตายเลย (Game Over)
                                    if round_number == 1:
                                        game_over = True
                                        # ไม่ต้องสั่งน้ำเพิ่ม เพราะอยากให้ตายทันที
                                    else:
                                        # บันทึกความยาวคำผิดเพื่อคำนวณ avg (ถาต้องการ)
                                        water.word_lengths.append(len(submitted_word))

                                        # ให้ขึ้นน้ำตามสูตร (avg * percent)
                                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                        avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                        water_rise_letters = avg_len * percent
                                        if water_rise_letters > 0:
                                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                            water.rise(rise_px)

                                        round_number += 1
                        # เคลียร์อินพุตเสมอหลังกด Enter
                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    # รีสตาร์ทเมื่อ game_over โดยกด R
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                        water = Water(start_level=HEIGHT - 100)
                        water.word_lengths = []
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False
                        submitted_words.clear()
                        info_text = ""
                        info_time = 0.0

        # --- อัปเดตน้ำ ---
        water.update(dt)

        # --- กล้อง (box follow) ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.12

        # --- ตรวจสอบ Game Over: เมื่อน้ำถึงบล็อกบนสุด (เช็คเมื่ออนิเมชัน water ถึง target แล้ว) ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if (not (water.level > water.target)) and (water.level <= top_block_y):
                game_over = True

        # --- วาดฉาก ---
        screen.fill(BLACK)

        # draw blocks
        for block in blocks:
            block.draw(screen, camera_y)

        # draw water
        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Category: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 220, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 220, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 220, 80)

        # draw info_text (temporary)
        if info_text:
            info_surf = FONT.render(info_text, True, YELLOW if "ถูกต้อง" in info_text else RED if "ผิด" in info_text or "ซ้ำ" in info_text else WHITE)
            screen.blit(info_surf, (20, 130))

        # game over overlay
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 19:40:55.425 Python[36378:13064928] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

In [ ]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 200)      # rgba-ish (used with SRCALPHA)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 28)
BIG = pygame.font.SysFont("arial", 56)

clock = pygame.time.Clock()

# --- คำถาม ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- Water class (easing + nicer waves) ---
class Water:
    def __init__(self, screen_h, start_level=None, anim_dur=0.9, color=BLUE):
        self.screen_h = screen_h
        self.level = float(start_level if start_level is not None else screen_h)  # world y
        # animation fields for smooth rising
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.anim_dur = anim_dur
        self.animating = False
        self.color = color

        # store correct word lengths only
        self.word_lengths = []

    def rise(self, amount_px):
        """Start a smooth rise by amount_px (pixels). Uses easing animation."""
        if amount_px <= 0:
            return
        self.start_y = float(self.level)
        self.target_y = max(0.0, self.start_y - float(amount_px))
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        """Update easing animation (smoothstep)."""
        if not self.animating:
            return
        self.anim_time += dt
        t = min(1.0, self.anim_time / self.anim_dur)
        # smoothstep easing
        t_ease = t * t * (3 - 2 * t)
        self.level = self.start_y + (self.target_y - self.start_y) * t_ease
        if t >= 1.0:
            self.level = self.target_y
            self.animating = False

    def draw(self, surface, camera_y=0):
        """Draw water with two-layer waves & subtle highlight."""
        screen_y = int(self.level - camera_y)
        height = max(0, self.screen_h - screen_y)
        if height <= 0:
            return

        # gradient surface: draw simple vertical gradient (top darker -> bottom lighter)
        grad = pygame.Surface((WIDTH, height), pygame.SRCALPHA)
        top_col = (25, 60, 160, 220)
        bottom_col = (60, 150, 230, 200)
        for i in range(height):
            # lerp color
            r = int(top_col[0] + (bottom_col[0] - top_col[0]) * (i / max(1, height - 1)))
            g = int(top_col[1] + (bottom_col[1] - top_col[1]) * (i / max(1, height - 1)))
            b = int(top_col[2] + (bottom_col[2] - top_col[2]) * (i / max(1, height - 1)))
            a = int(top_col[3] + (bottom_col[3] - top_col[3]) * (i / max(1, height - 1)))
            pygame.draw.line(grad, (r, g, b, a), (0, i), (WIDTH, i))
        surface.blit(grad, (0, screen_y))

        # wave layers (screen-space)
        t = pygame.time.get_ticks() / 1000.0
        # back wave: slower, bigger
        points_back = []
        amp_back = 8
        wave_len_back = 240
        speed_back = 0.6
        for x in range(0, WIDTH + 8, 8):
            y = screen_y + math.sin((x / wave_len_back) + t * speed_back) * amp_back
            points_back.append((x, y))
        if len(points_back) > 1:
            pygame.draw.lines(surface, (20, 60, 120), False, points_back, 2)

        # front wave: faster, smaller
        points_front = []
        amp_front = 4
        wave_len_front = 100
        speed_front = 1.4
        for x in range(0, WIDTH + 6, 6):
            y = screen_y + math.sin((x / wave_len_front) + t * speed_front) * amp_front - 2
            points_front.append((x, y))
        if len(points_front) > 1:
            pygame.draw.lines(surface, (180, 220, 255), False, points_front, 2)

        # highlight band (soft)
        hl_points = []
        for x in range(0, WIDTH + 6, 12):
            y = screen_y + math.sin((x / 60.0) + t * 2.0) * 2.0 - 6
            hl_points.append((x, y))
        if len(hl_points) > 1:
            s = pygame.Surface((WIDTH, 24), pygame.SRCALPHA)
            pygame.draw.lines(s, (255, 255, 255, 40), False, [(p[0], p[1] - screen_y + 6) for p in hl_points], 3)
            surface.blit(s, (0, screen_y - 12))

    def is_animating(self):
        return self.animating

    def reset(self, start_level=None):
        start = start_level if start_level is not None else self.screen_h
        self.level = float(start)
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.animating = False
        self.word_lengths = []


# --- ฟังก์ชันอื่น ๆ ---
blocks = []


def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง (stack)"""
    word = word.strip()
    if word == "":
        return

    # base top y for the lowest new block:
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height  # top y of the first (lowest) block
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height

    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""
    # เลือกหมวดเดียวสำหรับทั้งการเล่น (per run)
    chosen_category = random.choice(list(questions.keys()))
    current_question = questions[chosen_category]

    camera_y = 0
    camera_target_y = 0

    # Water (easing)
    water = Water(screen_h=HEIGHT, start_level=HEIGHT - 100, anim_dur=0.9)
    # 1 char -> 1 block high by default
    WATER_RISE_PIXELS_PER_LETTER = block_height

    # percent config (no cap)
    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10

    score = 0
    game_over = False

    # duplicate guard (case-insensitive)
    submitted_words = set()

    # temporary info text
    info_text = ""
    info_time = 0.0
    INFO_DURATION = 2.0

    while running:
        dt = clock.tick(60) / 1000.0

        # clear info text after duration
        if info_text and (pygame.time.get_ticks() / 1000.0 - info_time) > INFO_DURATION:
            info_text = ""

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # duplicate check (case-insensitive)
                            if submitted_word in submitted_words:
                                info_text = "Already used!"
                                info_time = pygame.time.get_ticks() / 1000.0
                                # do nothing else
                            else:
                                # correct?
                                if submitted_word in current_question:
                                    submitted_words.add(submitted_word)
                                    add_word_blocks(submitted_word)
                                    score += len(submitted_word) * 10

                                    # record correct lengths only
                                    water.word_lengths.append(len(submitted_word))

                                    # compute percent & rise (based on avg of correct words)
                                    percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                    # no cap as requested
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                    water_rise_letters = avg_len * percent
                                    if water_rise_letters > 0:
                                        rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                        water.rise(rise_px)

                                    round_number += 1

                                    info_text = "Correct!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                else:
                                    # wrong: do NOT include wrong into avg (per request)
                                    info_text = "Wrong!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    # if wrong on first round -> instant death
                                    if round_number == 1:
                                        game_over = True
                                    else:
                                        # still advance round (so percent grows each submission)
                                        # but WRONG does NOT append into water.word_lengths
                                        # To compute rise, use avg of correct words if available,
                                        # otherwise fallback to the wrong word length.
                                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)

                                        if len(water.word_lengths) > 0:
                                            avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                        else:
                                            # fallback: use the length of the wrong word
                                            avg_len = len(submitted_word)

                                        water_rise_letters = avg_len * percent
                                        if water_rise_letters > 0:
                                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                            water.rise(rise_px)

                                        round_number += 1

                        # clear input always
                        input_text = ""

                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    # restart with same behavior: pick a new category for the new run
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        chosen_category = random.choice(list(questions.keys()))
                        current_question = questions[chosen_category]
                        water.reset(start_level=HEIGHT - 100)
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False
                        submitted_words.clear()
                        info_text = ""
                        info_time = 0.0

        # update water (smooth easing)
        water.update(dt)

        # camera follow
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        # smooth camera
        camera_y += (camera_target_y - camera_y) * 0.12

        # game over check: when water finalized and reaches top block
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if (not water.is_animating()) and (water.level <= top_block_y):
                game_over = True

        # draw
        screen.fill(BLACK)

        for block in blocks:
            block.draw(screen, camera_y)

        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Category: {chosen_category}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 220, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 220, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 220, 80)

        # info text
        if info_text:
            # color selection
            color = YELLOW
            if info_text.lower().startswith("wrong") or info_text.lower().startswith("already"):
                color = RED
            draw_text(screen, info_text, FONT, color, 20, 130)

        # game over overlay
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 19:52:09.705 Python[36438:13077250] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

In [4]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 200)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)
WATER_COLOR = LIGHT_BLUE
WATER_SURFACE_LENGTH = 120
WATER_SURFACE_WAVE_AMPLITUDE = 6
WATER_ANIM_DURATION = 0.9

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 28)
BIG = pygame.font.SysFont("arial", 56)

clock = pygame.time.Clock()

# --- คำถาม (categories) ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- บล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- Water class (easing + nicer waves) ---
class Water:
    def __init__(self, screen_h, start_level=None, color=WATER_COLOR,
                 wave_amp=WATER_SURFACE_WAVE_AMPLITUDE,
                 wave_len=WATER_SURFACE_LENGTH,
                 anim_dur=WATER_ANIM_DURATION):
        self.screen_h = screen_h
        self.level_y = float(start_level if start_level is not None else screen_h)
        self.visible = False

        # animation (easing)
        self.animating = False
        self.start_y = float(self.level_y)
        self.target_y = float(self.level_y)
        self.anim_time = 0.0
        self.anim_dur = anim_dur

        # wave params
        self.base_amp = wave_amp
        self.base_len = wave_len
        self.color = color

    def start_rise(self, rise_pixels):
        """เริ่มให้ น้ำขึ้น `rise_pixels` (พิกเซล). รองรับการ stack ขณะ animating."""
        if not self.visible:
            self.visible = True
            # ensure float to avoid int truncation
            self.level_y = float(self.level_y)
        # start from current level (so stacking is smooth)
        self.start_y = float(self.level_y)
        # target cannot go above y=0
        self.target_y = max(0.0, self.start_y - float(rise_pixels))
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        """อัปเดตอนิเมชันการขึ้นแบบ smoothstep + small bounce at end."""
        if self.animating:
            self.anim_time += dt
            t = min(1.0, self.anim_time / self.anim_dur)
            # smoothstep easing (same as before)
            t_ease = t * t * (3.0 - 2.0 * t)
            # interpolate
            self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease

            # small "settle" bounce when finishing to feel heavier (subtle)
            if t >= 1.0:
                # make sure final pos is exact
                self.level_y = self.target_y
                self.animating = False

    def draw(self, surf):
        """วาดผืนน้ำเป็น surface โปร่งแสง + คลื่น 2 ชั้น + highlight"""
        if not self.visible:
            return

        level = int(self.level_y)
        if level >= self.screen_h:
            return  # completely off-screen

        # surface height in screen coords
        height = max(0, self.screen_h - level)
        water_surf = pygame.Surface((WIDTH, height), pygame.SRCALPHA)

        # fill base color (use the color tuple if given)
        try:
            water_surf.fill(self.color)
        except Exception:
            # fallback if color not RGBA
            water_surf.fill((30, 70, 200, 180))

        # draw wave layers in screen space (y coordinates relative to top of water_surf)
        t = pygame.time.get_ticks() / 1000.0

        # back wave (big, slow) - draws darker strokes slightly below surface
        pts_back = []
        amp_back = self.base_amp * 1.0
        len_back = self.base_len * 1.6
        speed_back = 0.6
        for x in range(0, WIDTH + 8, 8):
            y = (math.sin((x / len_back) + t * speed_back) * amp_back) + 12  # offset down a bit
            pts_back.append((x, y + 0))
        if len(pts_back) > 1:
            pygame.draw.lines(water_surf, (18, 45, 110, 180), False, pts_back, 2)

        # front wave (small, fast) - draws brighter strokes right at surface
        pts_front = []
        amp_front = self.base_amp * 0.55
        len_front = self.base_len * 0.6
        speed_front = 1.5
        for x in range(0, WIDTH + 6, 6):
            # modulate amplitude a bit by the rise speed to make surface choppy while rising
            rise_speed_factor = 1.0
            if self.animating and self.anim_time < self.anim_dur:
                # while animating, make the front wave a bit larger for energy
                rise_speed_factor += 0.6 * (1.0 - (self.anim_time / max(1e-6, self.anim_dur)))
            y = math.sin((x / len_front) + t * speed_front) * (amp_front * rise_speed_factor)
            pts_front.append((x, y))
        if len(pts_front) > 1:
            # draw light front wave
            pygame.draw.lines(water_surf, (200, 230, 255, 200), False, pts_front, 2)

        # highlight polygon along the top: build from pts_front but clamp near top
        highlight_poly = []
        for (x, y) in pts_front:
            # make highlight a few px above the computed front wave
            highlight_poly.append((x, max(0, y - 6)))
        if highlight_poly:
            # create a poly from left-top, the highlight curve, then right-top, then fill a soft triangular band
            poly = [(0, 0)] + [(x, y) for (x, y) in highlight_poly] + [(WIDTH, 0)]
            pygame.draw.polygon(water_surf, (255, 255, 255, 18), poly)

        # finally blit water surface on main surface at correct world Y
        surf.blit(water_surf, (0, level))

        # draw a stronger white crest line slightly offset (screen coords)
        crest_pts = [(x, int(level + y - 2)) for (x, y) in pts_front]
        if len(crest_pts) > 1:
            pygame.draw.lines(surf, (240, 250, 255), False, crest_pts, 2)

    def is_animating(self):
        return self.animating

    def reset(self):
        # reset to hidden bottom
        self.__init__(self.screen_h)



# --- helpers ---
blocks = []


def add_word_blocks(word):
    word = word.strip()
    if word == "":
        return
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height
    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""

    # choose ONE category for the whole run
    chosen_category = random.choice(list(questions.keys()))
    current_question = questions[chosen_category]

    camera_y = 0
    camera_target_y = 0

    water = Water(screen_h=HEIGHT, start_level=HEIGHT - 100, anim_dur=0.9)
    WATER_RISE_PIXELS_PER_LETTER = block_height

    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10

    score = 0
    game_over = False

    submitted_words = set()  # case-insensitive stored as lower()

    info_text = ""
    info_time = 0.0
    INFO_DURATION = 2.0

    while running:
        dt = clock.tick(60) / 1000.0

        if info_text and (pygame.time.get_ticks() / 1000.0 - info_time) > INFO_DURATION:
            info_text = ""

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # duplicate -> treat like wrong: no block add, but water rises
                            if submitted_word in submitted_words:
                                info_text = "Already used!"
                                info_time = pygame.time.get_ticks() / 1000.0

                                # compute rise just like wrong (do NOT append to correct-word list)
                                percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                # avg of correct words if exist, else fallback to length of duplicated word
                                if len(water.word_lengths) > 0:
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                else:
                                    avg_len = len(submitted_word)
                                water_rise_letters = avg_len * percent
                                if water_rise_letters > 0:
                                    rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                    water.rise(rise_px)
                                round_number += 1

                            else:
                                # not duplicate -> check correctness
                                if submitted_word in current_question:
                                    submitted_words.add(submitted_word)
                                    add_word_blocks(submitted_word)
                                    score += len(submitted_word) * 10

                                    # append only correct words to avg list
                                    water.word_lengths.append(len(submitted_word))

                                    percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                    water_rise_letters = avg_len * percent
                                    if water_rise_letters > 0:
                                        rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                        water.rise(rise_px)

                                    round_number += 1

                                    info_text = "Correct!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    # IMPORTANT: do NOT change category mid-run
                                else:
                                    # wrong (not duplicate): no append to avg
                                    info_text = "Wrong!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    if round_number == 1:
                                        # first-round wrong = instant death
                                        game_over = True
                                    else:
                                        # compute rise using avg if exists otherwise fallback to wrong len
                                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                        if len(water.word_lengths) > 0:
                                            avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                        else:
                                            avg_len = len(submitted_word)
                                        water_rise_letters = avg_len * percent
                                        if water_rise_letters > 0:
                                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                            water.rise(rise_px)
                                        round_number += 1

                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        chosen_category = random.choice(list(questions.keys()))
                        current_question = questions[chosen_category]
                        water.reset(start_level=HEIGHT - 100)
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False
                        submitted_words.clear()
                        info_text = ""
                        info_time = 0.0

        # update water animation
        water.update(dt)

        # camera follow
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.12

        # GAME OVER CHECK: immediate when water reaches top block (no need to wait animation)
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if water.level <= top_block_y:
                game_over = True

        # draw scene
        screen.fill(BLACK)
        for block in blocks:
            block.draw(screen, camera_y)
        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Category: {chosen_category}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 220, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 220, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 220, 80)

        # info text
        if info_text:
            color = YELLOW
            if info_text.lower().startswith("wrong") or info_text.lower().startswith("already"):
                color = RED
            draw_text(screen, info_text, FONT, color, 20, 130)

        # overlay when game over
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


TypeError: draw() takes 2 positional arguments but 3 were given

In [5]:
class Water:
    def __init__(self, screen_h, color=WATER_COLOR,
                 wave_amp=WATER_SURFACE_WAVE_AMPLITUDE,
                 wave_len=WATER_SURFACE_LENGTH,
                 anim_dur=WATER_ANIM_DURATION):
        self.screen_h = screen_h
        self.level_y = float(screen_h)   # world coordinate of water surface (y increases down)
        self.visible = False

        # animation (easing)
        self.animating = False
        self.start_y = float(self.level_y)
        self.target_y = float(self.level_y)
        self.anim_time = 0.0
        self.anim_dur = anim_dur

        # wave params
        self.base_amp = wave_amp
        self.base_len = wave_len
        self.color = color

    def start_rise(self, rise_pixels):
        """เริ่มให้ น้ำขึ้น `rise_pixels` (พิกเซล). รองรับการ stack ขณะ animating."""
        if not self.visible:
            self.visible = True
            # ensure float to avoid int truncation
            self.level_y = float(self.level_y)
        # start from current level (so stacking is smooth)
        self.start_y = float(self.level_y)
        # target cannot go above y=0
        self.target_y = max(0.0, self.start_y - float(rise_pixels))
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        """อัปเดตอนิเมชันการขึ้นแบบ smoothstep + small bounce at end."""
        if self.animating:
            self.anim_time += dt
            t = min(1.0, self.anim_time / self.anim_dur)
            # smoothstep easing (same as before)
            t_ease = t * t * (3.0 - 2.0 * t)
            # interpolate
            self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease

            # small "settle" bounce when finishing to feel heavier (subtle)
            if t >= 1.0:
                # make sure final pos is exact
                self.level_y = self.target_y
                self.animating = False

    def draw(self, surf):
        """วาดผืนน้ำเป็น surface โปร่งแสง + คลื่น 2 ชั้น + highlight"""
        if not self.visible:
            return

        level = int(self.level_y)
        if level >= self.screen_h:
            return  # completely off-screen

        # surface height in screen coords
        height = max(0, self.screen_h - level)
        water_surf = pygame.Surface((WIDTH, height), pygame.SRCALPHA)

        # fill base color (use the color tuple if given)
        try:
            water_surf.fill(self.color)
        except Exception:
            # fallback if color not RGBA
            water_surf.fill((30, 70, 200, 180))

        # draw wave layers in screen space (y coordinates relative to top of water_surf)
        t = pygame.time.get_ticks() / 1000.0

        # back wave (big, slow) - draws darker strokes slightly below surface
        pts_back = []
        amp_back = self.base_amp * 1.0
        len_back = self.base_len * 1.6
        speed_back = 0.6
        for x in range(0, WIDTH + 8, 8):
            y = (math.sin((x / len_back) + t * speed_back) * amp_back) + 12  # offset down a bit
            pts_back.append((x, y + 0))
        if len(pts_back) > 1:
            pygame.draw.lines(water_surf, (18, 45, 110, 180), False, pts_back, 2)

        # front wave (small, fast) - draws brighter strokes right at surface
        pts_front = []
        amp_front = self.base_amp * 0.55
        len_front = self.base_len * 0.6
        speed_front = 1.5
        for x in range(0, WIDTH + 6, 6):
            # modulate amplitude a bit by the rise speed to make surface choppy while rising
            rise_speed_factor = 1.0
            if self.animating and self.anim_time < self.anim_dur:
                # while animating, make the front wave a bit larger for energy
                rise_speed_factor += 0.6 * (1.0 - (self.anim_time / max(1e-6, self.anim_dur)))
            y = math.sin((x / len_front) + t * speed_front) * (amp_front * rise_speed_factor)
            pts_front.append((x, y))
        if len(pts_front) > 1:
            # draw light front wave
            pygame.draw.lines(water_surf, (200, 230, 255, 200), False, pts_front, 2)

        # highlight polygon along the top: build from pts_front but clamp near top
        highlight_poly = []
        for (x, y) in pts_front:
            # make highlight a few px above the computed front wave
            highlight_poly.append((x, max(0, y - 6)))
        if highlight_poly:
            # create a poly from left-top, the highlight curve, then right-top, then fill a soft triangular band
            poly = [(0, 0)] + [(x, y) for (x, y) in highlight_poly] + [(WIDTH, 0)]
            pygame.draw.polygon(water_surf, (255, 255, 255, 18), poly)

        # finally blit water surface on main surface at correct world Y
        surf.blit(water_surf, (0, level))

        # draw a stronger white crest line slightly offset (screen coords)
        crest_pts = [(x, int(level + y - 2)) for (x, y) in pts_front]
        if len(crest_pts) > 1:
            pygame.draw.lines(surf, (240, 250, 255), False, crest_pts, 2)

    def is_animating(self):
        return self.animating

    def reset(self):
        # reset to hidden bottom
        self.__init__(self.screen_h)


In [2]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 200)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 28)
BIG = pygame.font.SysFont("arial", 56)

clock = pygame.time.Clock()

# --- คำถาม (categories) ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- บล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- Water class (easing + nicer waves) ---
class Water:
    def __init__(self, screen_h, start_level=None, anim_dur=0.9, color=BLUE):
        self.screen_h = screen_h
        self.level = float(start_level if start_level is not None else screen_h)  # world y
        # animation fields for smooth rising
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.anim_dur = anim_dur
        self.animating = False
        self.color = color

        # store correct word lengths only
        self.word_lengths = []

    def rise(self, amount_px):
        """Start a smooth rise by amount_px (pixels). Uses easing animation."""
        if amount_px <= 0:
            return
        # allow stacking rises while animating: start from current level
        self.start_y = float(self.level)
        # ❌ เอา max(0.0, ...) ออก เพื่อให้ขึ้นเกินขอบบนได้
        self.target_y = self.start_y - float(amount_px)
        self.anim_time = 0.0
        self.animating = True


    def update(self, dt):
        """Update easing animation (smoothstep)."""
        if not self.animating:
            return
        self.anim_time += dt
        t = min(1.0, self.anim_time / self.anim_dur)
        # smoothstep easing
        t_ease = t * t * (3 - 2 * t)
        self.level = self.start_y + (self.target_y - self.start_y) * t_ease
        if t >= 1.0:
            self.level = self.target_y
            self.animating = False

    def draw(self, surface, camera_y=0):
        """Draw water with two-layer waves & subtle highlight."""
        screen_y = int(self.level - camera_y)
        height = max(0, self.screen_h - screen_y)
        if height <= 0:
            return

        # gradient surface
        grad = pygame.Surface((WIDTH, height), pygame.SRCALPHA)
        top_col = (25, 60, 160, 220)
        bottom_col = (60, 150, 230, 200)
        for i in range(height):
            r = int(top_col[0] + (bottom_col[0] - top_col[0]) * (i / max(1, height - 1)))
            g = int(top_col[1] + (bottom_col[1] - top_col[1]) * (i / max(1, height - 1)))
            b = int(top_col[2] + (bottom_col[2] - top_col[2]) * (i / max(1, height - 1)))
            a = int(top_col[3] + (bottom_col[3] - top_col[3]) * (i / max(1, height - 1)))
            pygame.draw.line(grad, (r, g, b, a), (0, i), (WIDTH, i))
        surface.blit(grad, (0, screen_y))

        # waves (back + front)
        t = pygame.time.get_ticks() / 1000.0
        points_back = []
        amp_back = 8
        wave_len_back = 240
        speed_back = 0.6
        for x in range(0, WIDTH + 8, 8):
            y = screen_y + math.sin((x / wave_len_back) + t * speed_back) * amp_back
            points_back.append((x, y))
        if len(points_back) > 1:
            pygame.draw.lines(surface, (20, 60, 120), False, points_back, 2)

        points_front = []
        amp_front = 4
        wave_len_front = 100
        speed_front = 1.4
        for x in range(0, WIDTH + 6, 6):
            y = screen_y + math.sin((x / wave_len_front) + t * speed_front) * amp_front - 2
            points_front.append((x, y))
        if len(points_front) > 1:
            pygame.draw.lines(surface, (180, 220, 255), False, points_front, 2)

        # highlight band (soft)
        hl_points = []
        for x in range(0, WIDTH + 6, 12):
            y = screen_y + math.sin((x / 60.0) + t * 2.0) * 2.0 - 6
            hl_points.append((x, y))
        if len(hl_points) > 1:
            s = pygame.Surface((WIDTH, 24), pygame.SRCALPHA)
            pygame.draw.lines(s, (255, 255, 255, 40), False, [(p[0], p[1] - screen_y + 6) for p in hl_points], 3)
            surface.blit(s, (0, screen_y - 12))

    def is_animating(self):
        return self.animating

    def reset(self, start_level=None):
        start = start_level if start_level is not None else self.screen_h
        self.level = float(start)
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.animating = False
        self.word_lengths = []


# --- helpers ---
blocks = []


def add_word_blocks(word):
    word = word.strip()
    if word == "":
        return
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height
    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""

    # choose ONE category for the whole run
    chosen_category = random.choice(list(questions.keys()))
    current_question = questions[chosen_category]

    camera_y = 0
    camera_target_y = 0

    water = Water(screen_h=HEIGHT, start_level=HEIGHT - 100, anim_dur=0.9)
    WATER_RISE_PIXELS_PER_LETTER = block_height

    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10

    score = 0
    game_over = False

    submitted_words = set()  # case-insensitive stored as lower()

    info_text = ""
    info_time = 0.0
    INFO_DURATION = 2.0

    while running:
        dt = clock.tick(60) / 1000.0

        if info_text and (pygame.time.get_ticks() / 1000.0 - info_time) > INFO_DURATION:
            info_text = ""

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # duplicate -> treat like wrong: no block add, but water rises
                            if submitted_word in submitted_words:
                                info_text = "Already used!"
                                info_time = pygame.time.get_ticks() / 1000.0

                                # compute rise just like wrong (do NOT append to correct-word list)
                                percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                # avg of correct words if exist, else fallback to length of duplicated word
                                if len(water.word_lengths) > 0:
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                else:
                                    avg_len = len(submitted_word)
                                water_rise_letters = avg_len * percent
                                if water_rise_letters > 0:
                                    rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                    water.rise(rise_px)
                                round_number += 1

                            else:
                                # not duplicate -> check correctness
                                if submitted_word in current_question:
                                    submitted_words.add(submitted_word)
                                    add_word_blocks(submitted_word)
                                    score += len(submitted_word) * 10

                                    # append only correct words to avg list
                                    water.word_lengths.append(len(submitted_word))

                                    percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                    water_rise_letters = avg_len * percent
                                    if water_rise_letters > 0:
                                        rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                        water.rise(rise_px)

                                    round_number += 1

                                    info_text = "Correct!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    # IMPORTANT: do NOT change category mid-run
                                else:
                                    # wrong (not duplicate): no append to avg
                                    info_text = "Wrong!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    if round_number == 1:
                                        # first-round wrong = instant death
                                        game_over = True
                                    else:
                                        # compute rise using avg if exists otherwise fallback to wrong len
                                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                        if len(water.word_lengths) > 0:
                                            avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                        else:
                                            avg_len = len(submitted_word)
                                        water_rise_letters = avg_len * percent
                                        if water_rise_letters > 0:
                                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                            water.rise(rise_px)
                                        round_number += 1

                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        chosen_category = random.choice(list(questions.keys()))
                        current_question = questions[chosen_category]
                        water.reset(start_level=HEIGHT - 100)
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False
                        submitted_words.clear()
                        info_text = ""
                        info_time = 0.0

        # update water animation
        water.update(dt)

        # camera follow
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.12

        # GAME OVER CHECK: immediate when water reaches top block (no need to wait animation)
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if water.level <= top_block_y:
                game_over = True

        # draw scene
        screen.fill(BLACK)
        for block in blocks:
            block.draw(screen, camera_y)
        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Category: {chosen_category}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 220, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 220, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 220, 80)

        # info text
        if info_text:
            color = YELLOW
            if info_text.lower().startswith("wrong") or info_text.lower().startswith("already"):
                color = RED
            draw_text(screen, info_text, FONT, color, 20, 130)

        # overlay when game over
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


KeyboardInterrupt: 